In [1]:
import numpy as np
import pandas as pd
import os
import random
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite

In [2]:
def decision(p):
    """
    Based on probability 0 <= p <= 1, determine decision of purchase
    
    """
    
    value = random.uniform(0,1)
    
    if value <= p:
        return True
    else:
        return False 
    

In [3]:
def getThreshold(player,players,properties):
    """
    This method returns the maximum of $200 and the maximum rent in the monopoly board at a given point in time
    """
    max_rent = 0
    for P in list(players.keys()):
        if P not in [player]:
            player_properties = players[P]['properties']
            for prop in list(player_properties.keys()):
                if 'Utility' in prop:
                    # Assume max possible rent is 10*dice_max = 10*12 = 120 (if player owns both utilities)
                    rent = 120
                elif 'Railroad' in prop:
                    railroad_count = 0
                    for prop_name in list(player_properties.keys()):
                        if "Railroad" in prop_name:
                            railroad_count += 1
                    
                    rent = 25*(2**(railroad_count-1))
                else:       
                    num_of_houses = player_properties[prop]['houses']
                    if num_of_houses == 0:
                        rent = properties[prop]["Rent"]
                    elif num_of_houses > 4:
                        rent = properties[prop]["Hotel"]
                    else:
                        rent = properties[prop][str(num_of_houses)]
                
                if rent > max_rent:
                    max_rent = rent
                    
    return max(200,max_rent)            
    

In [4]:
def sellProperties(player,players,properties,amount,min_bal):
    """
    Removes properties from ownership till player has enough balance required to pay the amount
    
        -> player: indicates the player whose balance needs to be checked
        -> amount: indicates the amount against which to be checked
        
    Player sells property back to the bank for 50% of total cost to build it
        
    """
    
    player_variables = players.get(player)
    
    property_names = list(player_variables["properties"].keys())
    property_values = []
    
    # Estimate total value of each property (with houses) 
    for prop_name in property_names:
        if "Railroad" in prop_name:
            price = properties[prop_name]["Price"]
            property_values.append(price)
        elif "Utility" in prop_name:
            price = properties[prop_name]["Price"]
            property_values.append(price)
        else :
            num_of_houses = players[player]["properties"][prop_name]["houses"]
            additional_cost = num_of_houses*properties[prop_name]["Price per house"]

            price = properties[prop_name]["Price"]

            total = price + additional_cost
            property_values.append(total)       
    
    while players[player]["balance"] < (min_bal + amount):        
        if len(property_values) == 0:
#             print("Sold all properties... still don't have enough balance :/")
#             print(f"Final balance: {players[player]['balance']}")
            return 0, players, properties # No properties remaining, player is STILL broke
        
        max_value, max_property = max(zip(property_values,property_names))
        players[player]["balance"] += 0.5*max_value 
        
        # Delete property name and value from local variables
        index = property_names.index(max_property) # get index based on name since price/value might not be unique
        property_names.pop(index)
        property_values.pop(index)
        
        # Delete property name and value from global variables
        del players[player]["properties"][max_property]
        del properties[max_property]["Owned by"]
        
        # Toggle property availablility to 1
        properties[max_property]["Availability"] = 1
        
#     print(f"Final balance: {players[player]['balance']}\n")
    
    return 1, players, properties # successful!
       

In [5]:
def canIBuildAHouse(player,players,properties):
    """
    This method checks if a player possesses sufficient balance to build a house each
    on all his properties ONLY if the player possesses AT LEAST 3 properties
    """
    player_variables = players.get(player)
    player_properties = player_variables.get("properties")
    
    if len(list(player_properties.keys())) >= 3:
        total_cost = 0
        for prop in list(player_properties.keys()):
            if ("Railroad" not in prop) and ("Utility" not in prop):
                num_of_houses = player_properties[prop]["houses"]
                if num_of_houses < 5:
                    house_cost = properties[prop]["Price per house"]
                    total_cost += house_cost

        if total_cost <= player_variables["balance"]:
            return 1
        else:
            return 0
    else:
        return 0       

In [6]:
def buildAHouse(player,players,properties):
    """
    This method builds houses on each of a player's properties
    """
    player_variables = players.get(player)
    player_properties = player_variables.get("properties")
    
    for prop in list(player_properties.keys()):
        if ("Railroad" not in prop) and ("Utility" not in prop):
            num_of_houses = player_properties[prop]["houses"]
            if num_of_houses < 5:
                price_per_house = properties[prop]["Price per house"]
                players[player]["balance"] -= price_per_house
                players[player]["properties"][prop]["houses"] += 1
                
    return players, properties
            

In [7]:
def estimateWealth(player,players,properties):
    """
    Returns total wealth of all assets for a given player at a given point in time
    """
    # DOES NOT MODIFY : NO NEED TO RETURN PLAYERS OR PROPERTIES
    
    cash = players[player]["balance"]
    
    props = players[player]["properties"]
    
    prop_value = 0
    
    if len(list(props.keys())) > 0:
        for prop in list(props.keys()):
            if ("Railroad" in prop) or ("Utility" in prop):
                prop_value += properties[prop]["Price"]
            else:
                num_of_houses = props[prop]["houses"]
                building_cost = num_of_houses*properties[prop]["Price per house"]
                price = properties[prop]["Price"] 

                prop_value += 0.5*(price + building_cost) # property is sold at 50% of original value BACK to the bank
        
    return (prop_value + cash)      
    

In [9]:
def initialize_properties():
    props =  {
             "Start" : 0, 
             "Mediterranean Avenue" : {"Price": 60, "Price per house": 50, "Rent": 2, "1": 10,"2": 30, "3": 90, "4": 160, "Hotel": 250, "Availability" : 1},
             "Baltic Avenue": {"Price": 60, "Price per house": 50, "Rent": 4, "1": 20,"2": 60, "3": 180, "4": 320, "Hotel": 450, "Availability" : 1},
             "Reading Railroad" : {"Price": 200, "Availability" : 1},
             "Oriental Avenue" : {"Price": 100, "Price per house": 50, "Rent": 6, "1": 30, "2": 90, "3": 270, "4": 400, "Hotel": 550, "Availability" : 1},
             "Vermont Avenue" : {"Price": 100, "Price per house": 50, "Rent": 6, "1": 30, "2": 90, "3": 270, "4": 400, "Hotel": 550, "Availability" : 1},
             "Connecticut Avenue" : {"Price": 120, "Price per house": 50, "Rent": 8, "1": 40, "2": 100, "3": 300, "4": 450, "Hotel": 600, "Availability" : 1},
             "St. Charles Place": {"Price": 140, "Price per house": 100, "Rent": 10, "1": 50, "2": 150, "3": 450, "4": 625, "Hotel": 750, "Availability" : 1},
             "Electric Company (Utility)" : {"Price": 150, "Availability" : 1},
             "States Avenue": {"Price": 140, "Price per house": 100, "Rent": 10, "1": 50, "2": 150, "3": 450, "4": 625, "Hotel": 750, "Availability" : 1},
             "Go to Jail" : 0,
             "Virginia Avenue" : {"Price": 160, "Price per house": 100, "Rent": 12, "1": 60, "2": 180, "3": 500, "4": 700, "Hotel": 900, "Availability" : 1},
             "Pennsylvania Railroad" : {"Price": 200, "Availability" : 1},
             "St. James Palace" : {"Price": 180, "Price per house": 100, "Rent": 14, "1": 70, "2": 200, "3": 550, "4": 750, "Hotel": 950, "Availability" : 1},
             "Tennessee Avenue" : {"Price": 180, "Price per house": 100, "Rent": 14, "1": 70, "2": 200, "3": 550, "4": 750, "Hotel": 950, "Availability" : 1},
             "New York Avenue" : {"Price": 200, "Price per house": 100, "Rent": 16, "1": 80, "2": 220, "3": 600, "4": 800, "Hotel": 1000, "Availability" : 1},
             "Kentucky Avenue" : {"Price": 220, "Price per house": 150, "Rent": 18, "1": 90, "2": 250, "3": 700, "4": 875, "Hotel": 1050, "Availability" : 1},
             "Indiana Avenue"  : {"Price": 220, "Price per house": 150, "Rent": 18, "1": 90, "2": 250, "3": 700, "4": 875, "Hotel": 1050, "Availability" : 1},
             "Illinois Avenue" : {"Price": 240, "Price per house": 150, "Rent": 20, "1": 100, "2": 300, "3": 750, "4": 925, "Hotel": 1100, "Availability" : 1},
             "B. & O. Railroad": {"Price": 200, "Availability" : 1},
             "Jail" : 0,
             "Atlantic Avenue" : {"Price": 260, "Price per house": 150, "Rent": 22, "1": 110, "2": 330, "3": 800, "4": 975, "Hotel": 1150, "Availability" : 1},
             "Ventnor Avenue" : {"Price": 260, "Price per house": 150, "Rent": 22, "1": 110, "2": 330, "3": 800, "4": 975, "Hotel": 1150, "Availability" : 1},
             "Water Works (Utility)" : {"Price": 150, "Availability" : 1},
             "Marvin Gardens" : {"Price": 280, "Price per house": 150, "Rent": 24, "1": 120, "2": 360, "3": 850, "4": 1025, "Hotel": 1200, "Availability" : 1},
             "Pacific Avenue" : {"Price": 300, "Price per house": 200, "Rent": 26, "1": 130, "2": 390, "3": 900, "4": 1100, "Hotel": 1275, "Availability" : 1},
             "North Carolina Avenue" : {"Price": 300, "Price per house": 200, "Rent": 26, "1": 130, "2": 390, "3": 900, "4": 1100, "Hotel": 1275, "Availability" : 1},
             "Pennsylvania Avenue" : {"Price": 320, "Price per house": 200, "Rent": 28, "1": 150, "2": 450, "3": 1000, "4": 1200, "Hotel": 1400, "Availability" : 1},
             "Short Line Railroad" : {"Price": 200, "Availability" : 1},
             "Park Place" : {"Price": 350, "Price per house": 200, "Rent": 35, "1": 175, "2": 500, "3": 1100, "4": 1300, "Hotel": 1500, "Availability" : 1},
             "Boardwalk" : {"Price": 400, "Price per house": 200, "Rent": 50, "1": 200, "2": 600, "3": 1400, "4": 1700, "Hotel": 2000, "Availability" : 1}
            }
    
    return props


In [10]:
def initialize_players(n):
    
    players = {}
    
    for i in range(1,n+1):
        player = f"P{i}"
        players[player] = {}
    
    for player in players:
        players[player]["balance"] = 1500
        players[player]["position"] = 0
        players[player]["purchase_odds"] = 1
        players[player]["go_to_jail"] = 0
        players[player]["jail_count"] = 0
        players[player]["active"] = 1
        players[player]["properties"] = {}
    
    return players

In [ ]:
def runSimulation(M,N,n):
    """
    M -> Number of simulations
    N -> Max. number of turns to be simulated (if game never ends) per simulation
    """
    game_end = []

    folder_path = f'./Simulation Results/M-{M} N-{N}'
    os.makedirs(folder_path, exist_ok=True)
    
    for sim_num in range(M):
        # Specify the folder and file name
        
        folder_2 = f'./Simulation Results/M-{M} N-{N}/Simulation {sim_num}/Wealth'
        folder_3 = f'./Simulation Results/M-{M} N-{N}/Simulation {sim_num}/Balances'

        # Create the directory if it doesn't exist
        
        os.makedirs(folder_2, exist_ok=True)
        os.makedirs(folder_3, exist_ok=True)
        
        if sim_num%(M/10)==0:
            print(sim_num)
            
        end_check = 0
#         stage_2 = 0
        
        properties = initialize_properties()
        players = initialize_players(n)

        player_count = len(list(players.keys()))
        
        player_wealth = {}
        for player in list(players.keys()):
            player_wealth.update({player : {}})

        # Initial dictionary for collecting property-wise landing frequency data 
        landing_frequencies = {}
        for prop in list(properties.keys()):
            landing_frequencies[prop] = 0

        balances = np.zeros((N,len(list(players.keys()))))
        
        jail_bail = 50
        income = 200
        
        all_property_check = 0
        for ROUND in range(1,N+1):
            bal = []
#             if player_count == 1:
#                 game_end.append(ROUND - 1)
#                 end_check = 1
#                 break
                
            for player in list(players.keys()):
                # Update wealth at the end of the round for each player
                player_wealth[player].update({ROUND : estimateWealth(player,players,properties)})
                
                # Check if player is still in the game 
                if(players[player]['active']):
                    bal.append(players[player]["balance"])

                    # Simulate TWO dice rolls
                    dice1 = random.randint(1,6) 
                    dice2 = random.randint(1,6)
                    dice = dice1 + dice2

                    # Get minimum balance threshold
                    min_balance = getThreshold(player,players,properties) # OK

                    l = len(list(properties.keys()))

                    current_location = list(properties.keys())[players[player]['position']] 

                    # **************************************** JAIL SEQUENCE ***********************************************
                    """
                    Logic for jail sequence (to be modified to this): DONE
                        -> If rolled doubles : GET OUT
                        -> Else, wait for 3 turns (if no doubles are rolled), THEN pay bail and get out!
                    """

                    player_check = 1
                    if (current_location == 'Jail'):
                        # Determine if player is just visiting or is in jail
                        if players[player]["go_to_jail"] == 1:
                            players[player]["jail_count"] += 1
                            if dice1==dice2:
                                # Player rolled doubles! He's out of jail for free!!!
                                players[player]["go_to_jail"] = 0
                                players[player]["jail_count"] = 0
                            else:
                                if players[player]["jail_count"] == 3:
                                    if players[player]["balance"] >= (jail_bail + min_balance):
                                        players[player]["balance"] -= jail_bail
                                        players[player]["go_to_jail"] = 0
                                        players[player]["jail_count"] = 0
                                    else:
                                        # Round 3 of staying in jail AND negative balance - sell property till enough balance
                                        # to pay the bail 
                                        players[player]["go_to_jail"] = 0
                                        players[player]["jail_count"] = 0
                                        decision, players, properties = sellProperties(player,players,properties,jail_bail,min_balance)
                                        if (decision):
                                            players[player]["balance"] -= jail_bail
                                        elif players[player]["balance"] >= jail_bail:
                                            # Player sold all their properties BUT still cannot maintain min_balance threshold
                                            # However, they may still possess sufficient balance pay the bail 
                                            # and make it to the next round!
                                            players[player]["balance"] -= jail_bail
                                        else:
                                            # player is broke!
                                            players[player]["balance"] = 0
                                            players[player]["active"] = 0
                                            player_count -= 1
                                            continue
                                else:
                                    continue               

                    # *************************************** END OF JAIL SEQUENCE ****************************************

                    # Update new position of player
                    oldPosition = players[player]['position']
                    newPosition = dice + players[player]['position'] # new position
                    players[player]['position'] = newPosition % l # normalize position w.r.t length of the board

                    # If player crosses over start point (position #0) add pre-set amount to player balance 
                    if newPosition >= l:
                        players[player]['balance'] += income

                    # Get property name of new location
                    toBuy = list(properties.keys())[players[player]['position']] 

                    # Update landing frequency dict
                    landing_frequencies[toBuy] += 1

                    if (toBuy == "Start"):
                        # player is in the "Start" square, skip the rest of the loop execution
                        continue

                    # If property is "Go to Jail", shift player to "Jail" square 
                    if (toBuy == 'Go to Jail'):
                        players[player]["position"] = (list(properties.keys())).index("Jail")
                        players[player]["go_to_jail"] = 1 # change status to 1 - indicates player is in jail, NOT visiting

                        # Now player needs to roll doubles OR wait 3 turns OR pay fine and get out of jail
                        # This is handled in the first few lines of the inner for loop (Jail Sequence)

                        continue # Skip to next player   

                    if (toBuy == "Jail"):
                        # player is visiting Jail, continue to next player
                        continue

                    """
                    Key goals of the next section: 
                        -> Check if property belongs to a player
                        -> If yes, pay rent 
                            -> If balance is not enough, sell properties till enough balance
                            -> If properties are exhausted, player is out of the game
                        -> If no, if enough balance: purchase, else: continue
                    """ 

                    property_variables = properties.get(toBuy)

                    if property_variables["Availability"] == 1:
                        if players[player]["balance"] >= (property_variables["Price"] + min_balance): #############
                            # Pay the price
                            players[player]["balance"] -= property_variables["Price"]

                            # Assign property to player with 0 houses (initially)
                            if ("Railroad" not in toBuy) and ("Utility" not in toBuy):
                                players[player]["properties"][toBuy] = {"houses" : 0} 
        #                         properties[toBuy] = {"Owned by" : player, "Availability" : 0}
                                properties[toBuy]["Availability"] = 0
                                properties[toBuy].update({"Owned by" : player})
                            else:
                                # Property is a Railroad OR Utility
                                # For now logic is the same in both if & else statements
                                # Can be modified later if required
                                players[player]["properties"][toBuy] = {"houses" : 0} 
                                properties[toBuy]["Availability"] = 0
                                properties[toBuy].update({"Owned by" : player})

                    else:
                        owner = property_variables["Owned by"] # get name of owner

                        owner_variables = players.get(owner) # get owner details

                        if player != owner:
                            if "Railroad" in toBuy:
                                # Railroad rent rules apply!
                                # 1x - $25, 2x - $50, 3x - $100, 4x - $200
                                railroad_count = 0
                                for prop in list(owner_variables['properties'].keys()):
                                    if "Railroad" in prop:
                                        railroad_count += 1

                                rent = 25*(2**(railroad_count-1))

                            elif "Utility" in toBuy:
                                # Utility rules apply
                                utility_count = 0 
                                for prop in list(owner_variables['properties'].keys()):
                                    if "Utility" in prop:
                                        utility_count += 1

                                if utility_count==1:
                                    rent = 4*dice
                                else:
                                    rent = 10*dice
                            else:
                                # Normal properties!
                                num_of_houses = owner_variables['properties'][toBuy]["houses"]

                                if num_of_houses==0:
                                    rent = property_variables["Rent"]
                                elif num_of_houses > 4:
                                    rent = property_variables["Hotel"]
                                else:
                                    rent = property_variables[str(num_of_houses)]

                            if players[player]["balance"] >= (rent + min_balance):
                                # Pay the rent!
                                players[player]["balance"] -= rent
                                players[owner]["balance"] += rent
                            else:
                                decision, players, properties = sellProperties(player,players,properties,jail_bail,min_balance)
                                if(decision):
                                    players[player]["balance"] -= rent
                                    players[owner]["balance"] += rent
                                elif players[player]["balance"] >= rent:
                                    # player sold ALL his properties, but he may STILL have enough balance 
                                    # to pay the bail
                                    players[player]["balance"] -= rent
                                    players[owner]["balance"] += rent
                                else: 
                                    # player is broke!
                                    players[player]["balance"] = 0
                                    players[player]["active"] = 0 # toggle player status to 0     
                                    player_count -= 1

                    # ******************* Last sequence: Build houses/hotels if enough balance post turn **********************

                    """
                    Here, we need to determine the logic that will be used to build houses. 
                    Let's first highlight key aspects of our approach here. 
                        -> Players will be allowed to start building houses once all properties are bought.
                        -> Only players with >= 3 properties can build houses.
                        -> Based on this logic, we are effectively modifying the rules of monopoly by allowing monopoly
                           to occur NOT based on colors, but simply by the number of properties owned by each player.

                    At this point, we need to determine the strategy that will be used by the players to build houses. 
                    In the paper we are referring to, monopoly is based on the original game wherein if a player owns 
                    all properties of a given color, they build houses on each of the properties at the same time (if 
                    they have enough money). 

                    We can use the same logic for our generalization for a "N"-player game. However, for smaller values
                    of N, this could be problematic since the chances of a player possessing more properties will be higher,
                    therefore, they will need to have more money to be able to build all houses at one go!
                    """   
                    
                    # Check if all properties have been bought
                    if all_property_check == 1:
#                         if stage_2 == 0:
#                             # Stage 2 begins when all properties are bought AND both players have at least $5000 in cash
#                             stage_2_check = 1
#                             for P in players:
#                                 if players[P]["balance"] < 5000:
#                                     stage_2_check = 0
#                                     break
                            
#                             if stage_2_check == 1:
#                                 stage_2 = ROUND
                            
                        # Let's build houses! (if I have at least 3 properties :/)
                        if (canIBuildAHouse(player,players,properties)):
                            players, properties = buildAHouse(player,players,properties)  
                        continue                         

                    # ******************************** CHECK IF ALL PROPERTIES HAVE BEEN BOUGHT ********************************

                    check = 1
                    for prop in list(properties.keys()):
                        default = ["Start","Go to Jail","Jail"]
                        if prop not in default:
                            if properties[prop]["Availability"] == 1:
                                check = 0
                                break

                    if check == 1:
                        all_property_check = 1          

                    """
                    Monopoly begins once all houses are bought. However, if a player leaves the game, there is a new phase
                    albeit a shorter one, of acquiring those properties by the remaining players. The simulation needs to 
                    accomodate for this phase as well.

                    """
                else:
                    bal.append(0)
                    
                

            for index in range(len(list(players.keys()))):
                balances[ROUND-1][index] = bal[index]
            
            if player_count == 1:
                game_end.append(ROUND)
                end_check = 1
                break

        if end_check == 0:
            # Game did NOT end in N rounds
            game_end.append(0)        
        
#         f'./Simulation Results/M-{M} N-{N}/Simulation {sim_num}/Balances'
        
        np.save(f"./Simulation Results/M-{M} N-{N}/Simulation {sim_num}/Balances/Simulation {sim_num+1}.npy", balances, allow_pickle=True, fix_imports=True)


        # Add code to save player_wealth information
        wealth = pd.DataFrame.from_dict(player_wealth)
        wealth.to_csv(f"./Simulation Results/M-{M} N-{N}/Simulation {sim_num}/Wealth/Simulation {sim_num+1}.csv")
       
    
    file_name = 'end_points.npy'
    
    # Save the numpy array to the specified path
    game_end = np.array(game_end)
    
    np.save(os.path.join(folder_path, file_name), game_end)
